In [1]:
import json
import os
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import requests
import ast

Read state folders with version 1.0

In [2]:
#state_folders = [item for item in os.listdir(os.getcwd()) if os.path.isdir(item) and item[-3:]=="1.0"]
#state_folders
state_folders = ["GAEEIOv1.0", "MEEEIOv1.0", "MNEEIOv1.0", "OREEIOv1.0", "WAEEIOv1.0"]

For each state we extract information from matrix.D for subsequent clustering, and reshape the 2D matrix into an 1D vector

In [3]:
data_all = []
for state_folder in state_folders:
    #json_file_path = os.path.join(state_folder, 'matrix', 'D.json')
    #with open(json_file_path, 'r') as file:
    #    data = json.load(file)
    url = f"https://raw.githubusercontent.com/ModelEarth/io/main/build/api/{state_folder}/matrix/D.json"
    data = requests.get(url).json()
    data_all.append(np.array(data).reshape(-1))
data_all = np.array(data_all)
data_all

array([[2.73512871e-02, 3.16932045e-04, 8.95749314e-04, ...,
        3.78456417e-01, 5.26837268e-01, 6.84648057e-01],
       [1.15993681e-02, 2.57649388e-03, 1.33494368e-03, ...,
        3.78535840e-01, 5.27454394e-01, 6.84647861e-01],
       [2.37501604e-02, 4.60245354e-03, 3.08355898e-05, ...,
        3.78508037e-01, 5.27398289e-01, 6.84647894e-01],
       [9.28698653e-03, 4.92553224e-04, 1.92236215e-02, ...,
        3.78536806e-01, 5.27483448e-01, 6.84647893e-01],
       [5.01933968e-03, 5.74106379e-04, 9.16535035e-05, ...,
        3.78557173e-01, 5.25713579e-01, 6.84648014e-01]])

In [4]:
data_all.shape

(5, 2336)

Reduce the dimension from 2336 to 3 of each vetorized D-matrix with PCA

In [5]:
from sklearn.decomposition import PCA
n_components = 3

# Create a PCA instance with the desired number of components
pca = PCA(n_components=n_components)

# Fit the PCA model and transform the original array
data_reduced = pca.fit_transform(data_all)

In [6]:
data_reduced

array([[-3.29386491e+02, -1.38102639e+02, -4.35914067e+01],
       [-3.54474781e+02, -1.23905093e+02, -1.26811229e+02],
       [-4.40413592e+02, -1.03079475e+02,  1.58609087e+02],
       [ 8.59300724e+02, -3.89594967e+02,  1.24827146e+01],
       [ 2.64974140e+02,  7.54682175e+02, -6.89165907e-01]])

Assigning 3 clusters, using the dimension-reduced vectorized D-matrix to cluster all these five states

In [7]:
from sklearn.cluster import KMeans
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(data_reduced)
cluster_labels = kmeans.labels_
print("Cluster labels:", cluster_labels)

Cluster labels: [0 0 0 2 1]


/Users/helix/Library/Data/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/helix/Library/Data/env/lib/python3.8/site-packages/threadpoolctl.py:1195: RuntimeWarning: libc not found. The ctypes module in Python 3.8 is maybe too old for this OS.
  warnings.warn(


In [8]:
cluster_result = dict(zip(state_folders, cluster_labels))
cluster_result

{'GAEEIOv1.0': 0,
 'MEEEIOv1.0': 0,
 'MNEEIOv1.0': 0,
 'OREEIOv1.0': 2,
 'WAEEIOv1.0': 1}